In [2]:
import pandas as pd
import numpy as np
import ast
import datetime as dt

In [3]:
data = pd.read_csv("C:/Users/Jose Quispe}/Desktop/SoyHenry/Proyecto Individual 1/movies_dataset.csv")

C:\Users\Jose Quispe}\AppData\Local\Temp\ipykernel_27896\3727508390.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:/Users/Jose Quispe}/Desktop/SoyHenry/Proyecto Individual 1/movies_dataset.csv")


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Transformaciones: Vas a hacer estas, y solo estas, transformaciones a los datos:

Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [4]:
# Funcion para convertir str no vacio a diccionario
def convertir_a_dicc(data_str):
    if pd.isna(data_str) or not isinstance(data_str, str) or data_str.strip() == "":
        return []
    else:
        return ast.literal_eval(data_str)

In [5]:
# Convertimos todas las columnas anidadas en diccionarios
df_belongs_to_collection = data['belongs_to_collection'].apply(convertir_a_dicc)
df_genres = data['genres'].apply(convertir_a_dicc)
df_production_companies = data['production_companies'].apply(convertir_a_dicc)
df_production_countries = data['production_countries'].apply(convertir_a_dicc)
df_spoken_lenguajes = data['spoken_languages'].apply(convertir_a_dicc)

In [6]:
# Definimos la función que procesa cada elemento
def extraer(x):
    if isinstance(x, list):  # Verifica si x es una lista
        return [d['name'] for d in x]  # Extrae los nombres
    else:
        return []  # Retorna lista vacía para otros tipos, como booleanos


In [7]:
# Nos quedamos solo con los datos que nos importan
df_genres = pd.DataFrame(df_genres.apply(extraer))
df_production_companies = pd.DataFrame(df_production_companies.apply(extraer))
df_production_countries = pd.DataFrame(df_production_countries.apply(extraer))
df_spoken_lenguajes = pd.DataFrame(df_spoken_lenguajes.apply(extraer))

# df_comedy = df[df['genres'].apply(lambda x: 'Comedy' in x)]


In [8]:
data = data.drop(columns=['belongs_to_collection', 'genres','production_companies','production_countries','spoken_languages'])

In [9]:
# Concatenamos con el dataframe del comienzo

df_final = pd.concat([data,
                      df_belongs_to_collection, 
                      df_genres, 
                      df_production_companies, 
                      df_production_countries,
                      df_spoken_lenguajes], axis=1)

In [9]:
df_final.head(1)

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,tagline,title,video,vote_average,vote_count,belongs_to_collection,genres,production_companies,production_countries,spoken_languages
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,NaN,Toy Story,False,7.7,5415.0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[Animation, Comedy, Family]",[Pixar Animation Studios],[United States of America],[English]


Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [10]:
df_final['revenue'] = df_final['revenue'].fillna(0)
df_final['budget'] = df_final['budget'].fillna(0)

Los valores nulos del campo release date deben eliminarse.


In [11]:
df_final = df_final.dropna(subset=['release_date'])

De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [12]:
def convertir_fecha(fecha_str):
    try:
        # Intentamos convertir a fecha con el formato especificado
        return pd.to_datetime(fecha_str, format='%Y-%m-%d', errors='raise')
    except (ValueError, TypeError):
        # Si falla, devolvemos NaT (valor nulo para fechas)
        return pd.NaT

In [13]:
# Aplicamos la función a la columna 'release_date'
df_final['release_date'] = df_final['release_date'].apply(convertir_fecha)

# Creamos la columna 'release_year' extrayendo el año, ignorando valores NaT
df_final['release_year'] = df_final['release_date'].dt.year

# Reemplazar NaT con cero si es necesario en 'release_year'
df_final['release_year'].fillna(0, inplace=True)


C:\Users\Jose Quispe}\AppData\Local\Temp\ipykernel_27896\1882727775.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final['release_year'].fillna(0, inplace=True)


Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [14]:
def calcular_return(row):
    try:
        budget = float(row['budget'])
        revenue = float(row['revenue'])
        if budget > 0:
            return revenue / budget
        else:
            return 0
    except ValueError:
        return 0  # En caso de que no se puedan convertir a números

df_final['return'] = df_final.apply(calcular_return, axis=1)

print(df_final[['revenue', 'budget', 'return']].head())


       revenue    budget     return
0  373554033.0  30000000  12.451801
1  262797249.0  65000000   4.043035
2          0.0         0   0.000000
3   81452156.0  16000000   5.090760
4   76578911.0         0   0.000000


Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [15]:
df_final = df_final.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])

In [16]:
import pyarrow

In [19]:

# Convertir la columna 'popularity' a tipo numérico
df_final['popularity'] = pd.to_numeric(df_final['popularity'], errors='coerce')
df_final = df_final.drop(columns=['belongs_to_collection'])
# Comprobar si hay NaN en la columna 'popularity' y tratar según sea necesario
nan_count = df_final['popularity'].isna().sum()
print(f"Número de valores NaN en 'popularity': {nan_count}")

if nan_count > 0:
    # Opción 1: Eliminar filas con NaN en 'popularity'
    df_final = df_final.dropna(subset=['popularity'])
    
    # Opción 2: Rellenar NaN con un valor por defecto (por ejemplo, 0)
    # df_final['popularity'].fillna(0, inplace=True)

# Verificar los tipos de datos antes de guardar
print(df_final.dtypes)

# Guardar el DataFrame en formato Parquet
df_final.to_parquet('df_final.parquet', index=False)

C:\Users\Jose Quispe}\AppData\Local\Temp\ipykernel_27896\3536565581.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['popularity'] = pd.to_numeric(df_final['popularity'], errors='coerce')


Número de valores NaN en 'popularity': 0
budget                          object
id                              object
original_language               object
overview                        object
popularity                     float64
release_date            datetime64[ns]
revenue                        float64
runtime                        float64
status                          object
tagline                         object
title                           object
vote_average                   float64
vote_count                     float64
genres                          object
production_companies            object
production_countries            object
spoken_languages                object
release_year                   float64
return                         float64
dtype: object
